In [ ]:
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install sklearn
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import plot_confusion_matrix
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import plot_confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import seaborn as sns

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [ ]:
train.head(20)

## ESPLORAZIONE BASICA DEL DATASET COMBATS PER VISUALIZZARE VALORI NULLI, VALORI RIPETUTI 

In [ ]:
feature = list(train)
print(feature)

In [ ]:
for c in train.columns:   
    print ("Features: ", c)
    print(train[c].value_counts())
    print("\n-------\n")

for c in test.columns:   
    print ("Features: ", c)
    print(test[c].value_counts())
    print("\n-------\n") 

#### OSSERVAZIONI:

##### * Utile stampare una rapida distribuzione, tuttavia analisi piu' accurate verranno fatte nella successiva sezione di Analisi Dei Dati

In [ ]:
print(train.isnull().sum()) #valori nulli in train
print(test.isnull().sum())  #valori nulli in test

#### OSSERVAZIONI:

##### * Non ci sono valori nulli nei dataset train e test derivanti dal file "combats.csv"

In [ ]:
train.info() #controllo i dtype

### TRASFORMO IL VALORE DI WINNER IN BINARIO, CIO' SERVIRA' SUCCESSIVAMENTE IN FASE DI CLASSIFICAZIONE

In [ ]:
def binary_winner(winner, first_pokemon, second_pokemon):
    if (first_pokemon == winner):
        return 0
    elif (second_pokemon == winner):
        return 1
    
train['Winner']= train.apply(lambda x: binary_winner(x['Winner'], x['First_pokemon'], x['Second_pokemon']), axis=1)
test['Winner']= test.apply(lambda x: binary_winner(x['Winner'], x['First_pokemon'], x['Second_pokemon']), axis=1)

In [ ]:
train.head(15)

## ESPLORAZIONE BASICA DEL DATASET POKEMON

In [ ]:
pokemonData = pd.read_csv('pokemon.csv')

In [ ]:
pokemonData.head()

In [ ]:
featurePokemon = list(pokemonData)
print(featurePokemon)

In [ ]:
for c in train.columns:   
    print ("Features: ", c)
    print(train[c].value_counts())
    print("\n-------\n")


#### OSSERVAZIONI:

##### * Utile stampare una rapida distribuzione, tuttavia analisi piu' accurate verranno fatte nella successiva sezione di Analisi Dei Dati

In [ ]:
print(pokemonData.isnull().sum()) #controllo i valori nulli

#### OSSERVAZIONI

##### * Valori nulli di type2 dovranno essere gestiti in seguito

In [ ]:
pokemonData.info() #controllo i dtype

## EFFETTUO IL JOIN TRA IL POKEMONDATA E IL TRAIN & TEST

In [ ]:
joinedTrain = train.join(pokemonData.set_index('#'), on="First_pokemon", rsuffix="_first")
train = joinedTrain.join(pokemonData.set_index('#'), on="Second_pokemon", rsuffix="_second")
joinedTest = test.join(pokemonData.set_index('#'), on="First_pokemon", rsuffix="_first")
test = joinedTest.join(pokemonData.set_index('#'), on="Second_pokemon", rsuffix="_second")

train.head() #visualizzo rapidamente il join compiuto

## ANALISI DEI DATI SUL DATASET JOINED

In [ ]:
print(train.isnull().sum()) #controllo i valori nulli dopo il join
print("\n-----------------SEPARATOR---------------\n")
print(test.isnull().sum()) #controllo i valori nulli dopo il join

#### OSSERVAZIONI:

##### * Gestiro' successivamente i valori nulli di type2 eliminando la colonna dato che 15mila valori nulli sono troppi per pensare di eliminare solo determinate righe 

##### * Per quanto riguarda i nomi dei Pokemon ci sono 10 valori nulli per i quali posso droppare direttamente le righe. Tuttavia i nomi dei Pokemon sono irrilevanti ai fini della classificazione stessa e per questo eliminero' direttamente tutta la colonna

In [ ]:
train.info() #stampo i DType

### EFFETTUO UN RAPIDO PREPROCESSING SULLE COLONNE CHE GIA' SI POSSONO ELIMINARE PER SCARSA RILEVANZA (NOME) E PER ECCESSO DI VALORI NULLI (TYPE 2)

In [ ]:
toDrop = ['Name','Name_second', 'Type 2', 'Type 2_second']
train = train.drop(toDrop, axis=1)
test = test.drop(toDrop, axis=1)

train.head()

In [ ]:
toEncode = ['Type 1','Legendary', 'Type 1_second', 'Legendary_second']

encoder = LabelEncoder()
for element in toEncode:
    train[element] = encoder.fit_transform(train[element])
    test[element] = encoder.fit_transform(test[element])

train.head()

### CONTROLLO LA DISTRIBUZIONE DEI VALORI E LA HEATMAP

In [ ]:
feature = list(train)
feature.remove('Winner')

for feature in feature:
    g = sns.displot(data=train, x=feature, hue=train['Winner'], kind="kde", facet_kws=dict(sharey=False, sharex=False))  
    plt.show()
    

#### OSSERVAZIONI:

##### * Possibile constatare dal grado di sovrapposizione della distribuzione dei valori winner che alcune feature sono completamente superflue alla classificazione, ad esempio generation, first_pokemon, second_pokemon, type 1, type 1_second

##### * Possibile constatare che alcune feature relative alle stats dei pokemon possiedono valori interessanti da quanto si osserva dalla distribuzione dei valori su winner, probabilmente e' possibile ricavare nuove feature computando la differenza tra le stats (ad esempio Attack, Defense, Sp. Attack, Sp. Defense, Speed ecc..). Lo provero' in seguito

In [ ]:
plt.figure(figsize=(18,10))
correlation = train.corr()
sns.heatmap(correlation, annot = True, linewidth = 2) #IN ALTERNATIVA POTEVO USARE BARPLOT E PANDA CORR

#### OSSERVAZIONI:

##### * La heatmap sostanzialmente conferma l'inutilita' dei valori generation, first_pokemon, second_pokemon, type 1 per entrambi i Pokemon dello scontro. Verranno sicuramente droppate in uno dei passaggi successivi

##### * Ora e' necessario verificare se l'aggiunta delle feature ipotizzate aggiunge informazione utile al fine della classifcazione

### SICCOME LE STATS SONO VALORI INTERI, POSSO PROVARE A VEDERE SE COMPUTARE LA DIFFERENZA TRA LE STATS DEI DUE POKEMON CI DA QUALCHE POSSIBILE VANTAGGIO NELLE HEATMAP

In [ ]:
##Aggiungo delle stats ulteriori
train['HP_Difference'] = train['HP'] - train['HP_second']
test['HP_Difference'] = test['HP'] - test['HP_second']
train['Attack_Difference'] = train['Attack'] - train['Attack_second']
test['Attack_Difference'] = test['Attack'] - test['Attack_second']
train['Defense_Difference'] = train['Defense'] - train['Defense_second']
test['Defense_Difference'] = test['Defense'] - test['Defense_second'] 
train['Sp. Atk_Difference'] = train['Sp. Atk'] - train['Sp. Atk_second']
test['Sp. Atk_Difference'] = test['Sp. Atk'] - test['Sp. Atk_second']
train['Sp. Def_Difference'] = train['Sp. Def'] - train['Sp. Def_second']
test['Sp. Def_Difference'] = test['Sp. Def'] - test['Sp. Def_second']
train['Speed_Difference'] = train['Speed'] - train['Speed_second']
test['Speed_Difference'] = test['Speed'] - test['Speed_second']

In [ ]:
newFeature = ['HP_Difference', 'Attack_Difference', 'Defense_Difference', 'Sp. Atk_Difference', 'Sp. Def_Difference', 'Speed_Difference']

plt.figure(figsize=(25,10))
for feature in newFeature:
    g = sns.displot(data=train, x=feature, hue=train['Winner'], kind="kde", facet_kws=dict(sharey=False, sharex=False))  
    plt.show()

### OSSERVAZIONI

##### * Queste nuove feature sembrano essere molto promettenti ad eccezione di Defense_Difference che e' troppo sovrapposta e "meno netta" nei valori di Winner, per confermare cio' e' possibile anche utilizzare una nuova heatmap

In [ ]:
plt.figure(figsize=(25,15))
correlation = train.corr()
sns.heatmap(correlation, annot = True, linewidth = 2)

#ANALIZZO PRINCIPALMENTE I CORRELATI A WINNER, INDIVIDUO VALORI POCO SIGNIFICATIVI CHE SONO QUELLI NELL'INTORNO DI 0:
#First_Pokemon, Second_pokemon, Type 1, Defense, Generation, Type 1_second, Defense_second, Generation_second, Defense_Difference

### OSSERVAZIONI:

##### * La heatmap conferma la scarsa rilevanza di Defense_Difference

##### * Le feature scartabili a seguito dell'analisi risultano essere: First_Pokemon, Second_pokemon, Type 1, Defense, Generation, Type 1_second, Defense_second, Generation_second, Defense_Difference

##### * Computare la differenza delle stats ha aiutato nella maggior parte dei casi con l'unica eccezione di Defense_Difference che continua ad avere una correlazione molto bassa

### SECONDO DROP DI VALORI POCO SIGNIFICATIVI

##### Possibile eliminare i seguenti valori per la bassa correlazione verificata in precedenza

In [ ]:
#Droppo i valori poco significativi analizzati in precedenza
toDrop = ['First_pokemon', 'Second_pokemon', 'Type 1', 'Defense', 'Generation', 'Type 1_second', 'Defense_second', 'Generation_second', 'Defense_Difference']
train = train.drop(toDrop, axis=1)
test = test.drop(toDrop, axis=1)

train.head()

# DEFINISCO LA PARTE DI TRAINING DEL MODELLO

In [ ]:
y_train = train['Winner']          
X_train = train.drop(['Winner'], axis=1)  

y_test = test['Winner']          
X_test = test.drop(['Winner'], axis=1)

In [ ]:
dt = DecisionTreeClassifier()
rf = RandomForestClassifier()

models = [dt, rf]
score = []

for model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = round(accuracy_score(y_pred, y_test) * 100, 2)
    score.append(acc)
    print(model)
    print(acc)
    plot_confusion_matrix(model, X_test, y_test, values_format='n', display_labels=['No','Yes']) 
    plt.show()
    print("\n")
    print(classification_report(y_test,y_pred))
    
models = pd.DataFrame({
    'Model': ['Decision Tree', 'Random Forest'],
    'Score': score})
models.sort_values(by='Score', ascending=False)

## OSSERVAZIONI CONCLUSIVE

#### * Random Forest: Anche se non spiegato a lezione, si tratta di un algoritmo che genera una moltitudine di decision tree. Ogni singolo albero nella foresta genera una previsione. Al termine si sceglie la classe di previsione che e' stata predetta per la maggiore tra gli alberi appartenenti alla foresta. Generalmente questo sistema "a maggioranza" risulta piu' performante del singolo decision tree

#### * Random Forest decisamente piu' performante sia del Decision Tree (come ci aspettavamo) di conseguenza e' l'algoritmo migliore per la classificazione

#### * Ho effettuato una rapida prova togliendo le feature "difference" aggiunte nella sezione di Analisi Di Dati ed ho constatato che un loro utilizzo migliora complessivamente il modello di classifcazione di 1 punto di Score

#### * Nel caso in cui non accettasse il Random Forest dato che non e' stato trattato a lezione, le ho lasciato anche il Decision Tree